In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from datasets import load_dataset
import pandas as pd
import torch
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
set_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False



In [ ]:
print(f"ha")

In [ ]:
from huggingface_hub import login

login("***REMOVED***")

In [ ]:
# model_name = "Qwen/Qwen2.5-3B-Instruct"
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

# model_name = "google/gemma-2-2b-it"
# file_qwen05_massive = "/kaggle/input/raw-qwen3-predictv1/raw_qwen3_predict_massive200.pt" #ini 3b
file_qwen05_massive = "/kaggle/input/raw-qwen05-predict-200/raw_qwen05_predict_massive200_selected_v.pt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# LAPE

In [ ]:
def LAPE(activation_probs, top_rate = 0.01,filter_rate=0.95,activation_bar_ratio=0.95):
    """    
    activation_probs = # layer x inter x lang_num
    """    
    num_layers = activation_probs.size(0)
    normed_activation_probs = activation_probs / activation_probs.sum(dim=-1, keepdim=True)
    normed_activation_probs[torch.isnan(normed_activation_probs)] = 0
    log_probs = torch.where(normed_activation_probs > 0, normed_activation_probs.log(), 0)
    entropy = -torch.sum(normed_activation_probs * log_probs, dim=-1)
    largest = False
    
    if torch.isnan(entropy).sum():
        print(torch.isnan(entropy).sum())
        raise ValueError
    
    flattened_probs = activation_probs.flatten()
    top_prob_value = flattened_probs.kthvalue(round(len(flattened_probs) * filter_rate)).values.item()
    print(top_prob_value)
    # dismiss the neruon if no language has an activation value over top 90%
    top_position = (activation_probs > top_prob_value).sum(dim=-1)
    entropy[top_position == 0] = -torch.inf if largest else torch.inf

    flattened_entropy = entropy.flatten()
    top_entropy_value = round(len(flattened_entropy) * top_rate)
    _, index = flattened_entropy.topk(top_entropy_value, largest=largest)
    row_index = index // entropy.size(1)
    col_index = index % entropy.size(1)
    selected_probs = activation_probs[row_index, col_index] # n x lang
    # for r, c in zip(row_index, col_index):
    #     print(r, c, activation_probs[r][c])

    # print(selected_probs.size(0), torch.bincount(selected_probs.argmax(dim=-1)))
    selected_probs = selected_probs.transpose(0, 1)
    activation_bar = flattened_probs.kthvalue(round(len(flattened_probs) * activation_bar_ratio)).values.item()
    # print((selected_probs > activation_bar).sum(dim=1).tolist())
    lang, indice = torch.where(selected_probs > activation_bar)

    merged_index = torch.stack((row_index, col_index), dim=-1)
    final_indice = []
    for _, index in enumerate(indice.split(torch.bincount(lang).tolist())):
        lang_index = [tuple(row.tolist()) for row in merged_index[index]]
        lang_index.sort()
        layer_index = [[] for _ in range(num_layers)]
        for l, h in lang_index:
            layer_index[l].append(h)
        for l, h in enumerate(layer_index):
            layer_index[l] = torch.tensor(h).long()
        final_indice.append(layer_index)
    return final_indice

def get_prob_for_lape(tensor, num_lang, num_layer, num_neurons):
    """
    num_neurons: intermediate layer (neurons in a layer)
    """
    full_languages_raw_values = (tensor.transpose(0,1)[0] > 0).half()
    probs = full_languages_raw_values.mean(dim=-2)
    probs.shape
    del full_languages_raw_values
    probs = probs.reshape(num_lang,num_layer,num_neurons)
    transposed_probs = probs.transpose(0,1).transpose(-1,-2)
    transposed_probs.shape
    return transposed_probs

def convert_to_global_indices(final_indice, intermediate_size):
    final_flattened = []
    
    for lang_idx, layers in enumerate(final_indice):  # Iterate over languages
        global_indices = []
        for layer_idx, hidden_units in enumerate(layers):  # Iterate over layers
            if hidden_units.numel() > 0:  # If there are selected neurons
                global_indices.extend((layer_idx * intermediate_size + hidden_units).tolist())
        
        final_flattened.append((global_indices))  
    
    return final_flattened



In [ ]:
import numpy as np
from scipy.stats import entropy
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


def get_k_lang_actv_dict(k, full_neurons, method="default", topk=0):
    
    """
    di sini anggap activated neuron (avg token) yang di setiap row di dataset(cuman 2 row) actv valuenya > 0.
    k = num language"""
    activation_dict = {}
    
    full_neurons = full_neurons.transpose(-3,-4)[0].transpose(-1,-2)

    if method == "default":
        for i in range (full_neurons.size(0)):
            tensor_lang = full_neurons[i]
            rows_with_both_positive = (tensor_lang > 0).all(dim=-1)
            
            indices = torch.where(rows_with_both_positive)[0]
            activation_dict[i] = indices
            # indices.shape
            # print(indices.shape)
    elif method == "topk":
        if topk==0:
            print(f"topk must not be 0")
        top = (full_neurons.mean(dim=-1).topk(topk).indices)
        for i in range (full_neurons.size(0)):
            activation_dict[i] = top[i]
    
    return activation_dict

# Step 1: Convert lists to probability distributions
def to_probability_distribution(values):
    total = sum(values)
    return [v / total for v in values]


def make_heatmap_neuron_overlap(activation_dict, k, with_label=True, method="default", alpha=1, normalized =False):    
    # Example dictionary: keys 0-52, values are 1D tensors of activated neuron indices
    # activation_dict = get_k_lang_actv_dict(10)

    overlap_matrix = torch.tensor([])
    if method == "default":
        # Step 1: Create a binary matrix
        max_neuron_index = max(max(indices) for indices in activation_dict.values()) + 1  # Find the maximum neuron index
        binary_matrix = torch.zeros((k+1, max_neuron_index), dtype=torch.int)  # Initialize binary matrix
        
        for key, indices in activation_dict.items():
            binary_matrix[key, indices] = 1  # Set activated neurons to 1
        
        # Step 2: Compute overlaps (dot product between rows)
        overlap_matrix = torch.matmul(binary_matrix, binary_matrix.T)  # Dot product of binary_matrix with its transpose

    elif method == "jaccard":
        max_neuron_index = max(max(indices) for indices in activation_dict.values()) + 1
        binary_matrix = torch.zeros((k+1, max_neuron_index), dtype=torch.int)
    
        # Fill binary matrix with activation data
        for key, indices in activation_dict.items():
            binary_matrix[key, indices] = 1  
    
        # Compute Jaccard distance matrix
        overlap_matrix = torch.zeros((k+1, k+1))
    
        for i in range(k+1):
            for j in range(k+1):
                intersection = (binary_matrix[i] & binary_matrix[j]).sum().item()
                union = (binary_matrix[i] | binary_matrix[j]).sum().item()
                jaccard_similarity = intersection / union if union > 0 else 0
                overlap_matrix[i, j] = jaccard_similarity
        overlap_matrix = overlap_matrix ** alpha
        if normalized:
            overlap_matrix = overlap_matrix / overlap_matrix.sum(axis=1, keepdims=True)

    
    # Step 3: Visualize the heatmap
    plt.figure(figsize=(10, 8))
    if with_label:
        sns.heatmap(overlap_matrix.numpy(), annot=True, fmt="d", cmap="YlOrRd", 
                    xticklabels=range(k+1), yticklabels=range(k+1))
    else:
        sns.heatmap(overlap_matrix.numpy(), fmt="d", cmap="YlOrRd",cbar=True)
    plt.xlabel("Key")
    plt.ylabel("Key")
    plt.title(f"Overlap Heatmap of Activated Neurons: {method}")
    plt.show()
    return overlap_matrix


def normed_heatmap_neuron_overlap(num_lang, activation_dict):
    # Step 1: Create a binary matrix
    max_neuron_index = max(max(indices) for indices in activation_dict.values()) + 1  # Find the maximum neuron index
    binary_matrix = torch.zeros((num_lang, max_neuron_index), dtype=torch.int)  # Initialize binary matrix
    
    for key, indices in activation_dict.items():
        binary_matrix[key, indices] = 1  # Set activated neurons to 1
    
    # Step 2: Compute overlaps (dot product between rows)
    overlap_matrix = torch.matmul(binary_matrix, binary_matrix.T)  # Dot product of binary_matrix with its transpose
    
    # Step 3: Normalize the overlap matrix
    # Compute the number of activated neurons for each key
    num_activated_neurons = binary_matrix.sum(dim=1, keepdim=True)
    
    # Normalize by the minimum number of activated neurons between each pair of keys
    normalized_overlap_matrix = overlap_matrix / torch.minimum(
        num_activated_neurons, num_activated_neurons.T
    )
    
    # Ensure the diagonal is exactly 1 (self-overlap is always 1)
    normalized_overlap_matrix.fill_diagonal_(1)
    
    # Step 4: Visualize the normalized heatmap (no labels or annotations)
    plt.figure(figsize=(10, 8))
    sns.heatmap(normalized_overlap_matrix.numpy(), cmap="YlOrRd", cbar=True, vmin=0, vmax=1)
    plt.xlabel("Key")
    plt.ylabel("Key")
    plt.title("Normalized Overlap Heatmap of Activated Neurons")
    plt.show()


def visualize_lape(full_languages_raw_values, num_layer, top_rate = 0.05,filter_rate=0.80,activation_bar_ratio=0.80): 
    num_lang, _, num_rows, num_neurons = full_languages_raw_values.shape
    
    per_layer = int(num_neurons/num_layer)
    transposed_probs = get_prob_for_lape(full_languages_raw_values, num_lang, num_layer, per_layer)
    lape = LAPE(transposed_probs, top_rate = top_rate,filter_rate=filter_rate,activation_bar_ratio=activation_bar_ratio)
    flattened_indices = convert_to_global_indices(lape, per_layer)
    activation_dict = dict()
    for i in range (num_lang):
        activation_dict[i] = flattened_indices[i]
    make_heatmap_neuron_overlap(activation_dict, num_lang, False, "jaccard", 1)
    return lape

In [ ]:
languages = {
    0: 'Afrikaans (af-ZA)',
    1: 'Amharic (am-ET)',
    2: 'Arabic (ar-SA)',
    3: 'Azerbaijani (az-AZ)',
    4: 'Bengali (bn-BD)',
    5: 'Catalan (ca-ES)',
    6: 'Welsh (cy-GB)',
    7: 'Danish (da-DK)',
    8: 'German (de-DE)',
    9: 'Greek (el-GR)',
    10: 'English (en-US)',
    11: 'Spanish (es-ES)',
    12: 'Persian (fa-IR)',
    13: 'Finnish (fi-FI)',
    14: 'French (fr-FR)',
    15: 'Hebrew (he-IL)',
    16: 'Hindi (hi-IN)',
    17: 'Hungarian (hu-HU)',
    18: 'Armenian (hy-AM)',
    19: 'Indonesian (id-ID)',
    20: 'Icelandic (is-IS)',
    21: 'Italian (it-IT)',
    22: 'Japanese (ja-JP)',
    23: 'Javanese (jv-ID)',
    24: 'Georgian (ka-GE)',
    25: 'Khmer (km-KH)',
    26: 'Kannada (kn-IN)',
    27: 'Korean (ko-KR)',
    28: 'Latvian (lv-LV)',
    29: 'Malayalam (ml-IN)',
    30: 'Mongolian (mn-MN)',
    31: 'Malay (ms-MY)',
    32: 'Burmese (my-MM)',
    33: 'Norwegian Bokmål (nb-NO)',
    34: 'Dutch (nl-NL)',
    35: 'Polish (pl-PL)',
    36: 'Portuguese (pt-PT)',
    37: 'Romanian (ro-RO)',
    38: 'Russian (ru-RU)',
    39: 'Slovenian (sl-SL)',
    40: 'Albanian (sq-AL)',
    41: 'Swedish (sv-SE)',
    42: 'Swahili (sw-KE)',
    43: 'Tamil (ta-IN)',
    44: 'Telugu (te-IN)',
    45: 'Thai (th-TH)',
    46: 'Tagalog (tl-PH)',
    47: 'Turkish (tr-TR)',
    48: 'Urdu (ur-PK)',
    49: 'Vietnamese (vi-VN)',
    50: 'Chinese (Simplified) (zh-CN)',
    51: 'Chinese (Traditional) (zh-TW)'
}

In [ ]:
import torch
full_languages_raw_values = torch.load(file_qwen05_massive, weights_only=False)


In [ ]:
# lape_massive_qwen_2 = visualize_lape(full_languages_raw_values, 24, top_rate = 0.02,filter_rate=0.80,activation_bar_ratio=0.80)
# lape_langs = {key:lape_massive_qwen_2[key] for key, value in enumerate(lape_massive_qwen_2)}


In [ ]:
# lape_langs = {key:lape_massive_qwen_2[key] for key, value in enumerate(lape_massive_qwen_2)}


In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
set_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

class InferenceModel:
    def __init__(self, model_name, device):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", offload_buffers=True)
    def inference(self, prompt, max_new_tokens=1):
        """
        Performs inference on a given prompt.
        Returns the decodede output
        """
        generated_text = ''
        len_sentence = 0
        if self.model_name == 'Qwen/Qwen2.5-0.5B-Instruct' or self.model_name == 'Qwen/Qwen2.5-3B-Instruct':
            # WARNING: messages ini harus sama dengan messages di check_index_prompt
            messages = [
            {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
            {"role": "user", "content": prompt}
            ]
            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            model_inputs = self.tokenizer([text], return_tensors="pt").to(self.model.device)
            
            generated_ids = self.model.generate(
                    **model_inputs,
                    do_sample=False,
                    temperature=None,
                    top_p=None,
                    top_k=None,
                    max_new_tokens=max_new_tokens
                )
            generated_ids = [
                output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
            ]
            generated_text = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
            len_sentence = len(self.tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)["input_ids"])
        elif self.model_name == "google/gemma-2-2b-it":
            # WARNING: messages ini harus sama dengan messages di check_index_prompt
            messages = [
            {"role": "user", "content": prompt}
            ]
            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            model_inputs = self.tokenizer([text], return_tensors="pt", add_special_tokens=False).to(self.model.device)
            # print(f"(model_inputs['input_ids'] {(model_inputs['input_ids'])}")
            
            # print(f"len(model_inputs['input_ids'] {len(model_inputs['input_ids'])}")
            sentence_token_texts = self.tokenizer.convert_ids_to_tokens((model_inputs['input_ids'])[0])
            # print(f"sentence_token_texts {sentence_token_texts}")
            
            
            generated_ids = self.model.generate(
                    **model_inputs,
                    do_sample=False,
                    temperature=None,
                    top_p=None,
                    top_k=None,
                    max_new_tokens=max_new_tokens
                )
            generated_ids = [
                output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
            ]
            generated_text = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
            len_sentence = len(self.tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)["input_ids"])
            # print()
            
        else:
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
            len_sentence = len(inputs["input_ids"])
            output = self.model.generate(
                **inputs, 
                do_sample=False,
                temperature=None,
                top_p=None,
                top_k=None,
                max_new_tokens=1)
            # print(f"output: {output}")
            generated_ids = [
                output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, output)
            ]
            generated_text = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        # answer = "1" if "option1" in generated_text else "2" if "option2" in generated_text else None
        
        return generated_text, len_sentence




In [ ]:
model = InferenceModel(model_name, device)

In [ ]:
def clean_hooks(infer_model):
    for i in range(len(infer_model.model.model.layers)):
        mlp = infer_model.model.model.layers[i].mlp
        mlp.act_fn._forward_hooks.clear()
def test_inference(infer_model, prompt, max_new_tokens):
    generated_text = infer_model.inference(prompt, max_new_tokens)
    # print(f"data: {prompt}")
            
    print(f"gen_text {generated_text}")
def set_activation_mlp(model_name, name, lape_lang, operation): 
    """
        name (str): buat namain layer
        lape_lang: lape untuk suatu lang
        operation: (bool) true if multiplied else replaced
    """
    operand = operation[0]
    replace_value = int(operation[1:])
    if operand not in ["=", "*", "+"]:
        raise ValueError("operand is wrong!")

    # TODO: coba di bagian promptnya aja jangan chat template
    def hook_fn(module, input, output):
        # print(f"output {output.shape}")
        start_id_to_intv = -3 if model_name == "Qwen/Qwen2.5-0.5B-Instruct" or model_name == 'Qwen/Qwen2.5-3B-Instruct' else 0 
        layer = int(name)
        if operand == "*":
            output[0, start_id_to_intv:, lape_lang[layer]] *= replace_value
        elif operand == "=":
            output[0, start_id_to_intv:, lape_lang[layer]] = replace_value
        else:
            output[0, start_id_to_intv:, lape_lang[layer]] += replace_value
    return hook_fn
def set_activation_mlp_v2(replace_method, model_name, name, lape_langs, target_lang, operation_non_target, operation_target): 
    """
    This changes all neuron lape values to be replaced_values but leave behind a desired target language. 
        replace_method: lape or all
        name (str): buat namain layer
        lape_langs: dict semua language lape
        target_lang: int index target_lang in lape
        operation: *10, =0, +5, etc
    """
    operand_t = operation_target[0]
    replace_value_t = int(operation_target[1:])
    if operand_t not in ["=", "*", "+"]:
        raise ValueError("operand is wrong!")

    operand_nt = operation_non_target[0]
    replace_value_nt = int(operation_non_target[1:])
    if operand_nt not in ["=", "*", "+"]:
        raise ValueError("operand is wrong!")

    # TODO: coba di bagian promptnya aja jangan chat template
    def hook_fn(module, input, output):
        # print(f"output {output.shape}")
        start_id_to_intv = -3 if model_name == "Qwen/Qwen2.5-0.5B-Instruct" or model_name == 'Qwen/Qwen2.5-3B-Instruct' else 0 
        layer = int(name)
        if replace_method == "lape":
            for lang, lape_lang in lape_langs.items():
                if lang == target_lang:
                    if operand_t == "*":
                        output[0, start_id_to_intv:, lape_lang[layer]] *= replace_value_t
                    elif operand_t == "=":
                        output[0, start_id_to_intv:, lape_lang[layer]] = replace_value_t
                    else:
                        output[0, start_id_to_intv:, lape_lang[layer]] += replace_value_t
                else:
                    if operand_nt == "*":
                        output[0, start_id_to_intv:, lape_lang[layer]] *= replace_value_nt
                    elif operand_nt == "=":
                        output[0, start_id_to_intv:, lape_lang[layer]] = replace_value_nt
                    else:
                        output[0, start_id_to_intv:, lape_lang[layer]] += replace_value_nt
                    
        # elif replace_method == "all":
        #     lape_target_lang = lape_langs[target_lang]
        #     if is_multiplied:
        #         output[0, start_id_to_intv:, (torch.arange(output.shape[-1]) != lape_target_lang[layer]).nonzero(as_tuple=True)[0]] *= replace_value
        #     else: 
        #         output[0, start_id_to_intv:, (torch.arange(output.shape[-1]) != lape_target_lang[layer]).nonzero(as_tuple=True)[0]]  = replace_value
                
    return hook_fn

def intervensi(prompt, infer_model, lape_lang, num_layers, max_new_tokens, operation):
    clean_hooks(infer_model)
    handlers = []
    for i in range (num_layers):
        mlp = infer_model.model.model.layers[i].mlp
        handlers.append(mlp.act_fn.register_forward_hook(set_activation_mlp(infer_model.model_name, f"{i}", lape_lang, operation)))
    test_inference(infer_model, prompt, max_new_tokens)
    for i in handlers:
        i.remove()
    clean_hooks(infer_model)

def intervensi_w_target_lang(replace_method, prompt, infer_model, lape_langs, target_lang, num_layers, max_new_tokens, operation_non_target, operation_target, range_layers):
    
    clean_hooks(infer_model)
    handlers = []
    assert range_layers[-1] <= num_layers
    for i in (range_layers):
        mlp = infer_model.model.model.layers[i].mlp
        handlers.append(mlp.act_fn.register_forward_hook(set_activation_mlp_v2(replace_method, infer_model.model_name, f"{i}", lape_langs, target_lang, operation_non_target, operation_target)))
    test_inference(infer_model, prompt, max_new_tokens)
    for i in handlers:
        i.remove()
    clean_hooks(infer_model)
    
def intervene_langs_w_target(
    replace_method, prompt, infer_model, lape_langs, num_layers, 
    target_lang=None, max_new_tokens=30, operation_non_target="=0", operation_target="*1", range_layers=range(1)):
    """
    target_lang: default None, kalau ada berarti cuman lihat di bahasa target_lang itu aja else semuanya 
    """
    print(f"🐸original: ")
    test_inference(infer_model, prompt, max_new_tokens)
    if target_lang:
        print(f"🐸 intervened w.r.t. a target language {languages[target_lang]}: ")
        intervensi_w_target_lang(replace_method, prompt, infer_model, lape_langs, target_lang, num_layers, max_new_tokens, operation_non_target, operation_target, range_layers)
    else:
        for key, value in lape_langs.items():
            print(f"🐸 intervened w.r.t. a target language {languages[key]}: ")
            intervensi_w_target_lang(replace_method, prompt, infer_model, lape_langs, key, num_layers, max_new_tokens, operation_non_target, operation_target, range_layers)

    clean_hooks(infer_model)
def intervene_langs(infer_model, prompt_lang, lape_langs, num_layers, max_new_tokens=10, operation="=0"):
    """
    given a prompt, from number of languages, see how intervention each language affects the prompt.
    infer_model: InferenceModel
    prompt_lang: (str) prompt dalam bahasa tertentu
    lape_langs: (dict) key: lang, value: lape in a language
    """
    print(f"🐸original: ")
    test_inference(infer_model, prompt_lang, max_new_tokens)
    for key, value in lape_langs.items():
        print(f"🐸intervensi {languages[key]}")
        intervensi(prompt_lang, infer_model, value, num_layers, max_new_tokens, operation)


Notes:

- di TransformerLens, `model.to_tokens(text)` sama dengan `model.tokenizer(text, return_tensors="pt")["input_ids"].to(device)` ini yg dimasukin ke model(). text harus berupa hasil tokenizer pake apply_chat_template (`text = model.tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)`). 

In [ ]:
# import torch

# def get_token_sequence_rank(model, messages, target_text, device="cuda"):
#     """
#     Get the rank of a multi-token sequence in the model's output probabilities.

#     Args:
#         model: The Hugging Face model (Qwen or similar).
#         messages (list): The chat messages in OpenAI format.
#         target_text (str): The full text whose token sequence rank we want to find.
#         device (str): The device to run the model on ("cuda" or "cpu").

#     Returns:
#         avg_rank (float): The average rank of the tokens (lower is better).
#         avg_probability (float): The geometric mean of token probabilities.
#         token_ranks (list): The individual ranks of each token in the sequence.
#         token_probs (list): The probability of each token.
#     """
#     # Convert messages to full text
#     text = model.tokenizer.apply_chat_template(
#         messages,
#         tokenize=False,
#         add_generation_prompt=True
#     )

#     # Tokenize input and target text
#     filled_input_ids = model.tokenizer(text, return_tensors="pt")["input_ids"].to(device)
#     target_token_ids = model.tokenizer(target_text, add_special_tokens=False)["input_ids"]
#     print(target_token_ids)
#     # Get logits from the model
#     logits = model.model(filled_input_ids, remove_batch_dim=True).logits

#     token_ranks = []
#     token_probs = []

#     for i, token_id in enumerate(target_token_ids):
#         # Compute softmax probabilities for token position `-len(target_token_ids) + i`
#         probs = torch.softmax(logits[:, -(len(target_token_ids) - i), :], dim=-1)

#         # Sort token probabilities in descending order
#         sorted_indices = torch.argsort(probs, descending=True)

#         # Find the rank of the token
#         target_rank = (sorted_indices == token_id).nonzero(as_tuple=True)[1]

#         if target_rank.numel() == 0:
#             rank = -1  # Token not found
#             probability = 0.0
#         else:
#             rank = target_rank.item() + 1  # Convert to 1-based rank
#             probability = probs[0, token_id].item()

#         token_ranks.append(rank)
#         token_probs.append(probability)

#     # Compute aggregate ranking and probability
#     avg_rank = sum(token_ranks) / len(token_ranks)
#     avg_probability = torch.prod(torch.tensor(token_probs)).item()  # Geometric mean

#     return avg_rank, avg_probability, token_ranks, token_probs
    
# prompts = ["What is the capital city of England?"]

# messages = [
#     {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
#     {"role": "user", "content": prompts[0]}
# ]

# target_text = "The "  # Can be a multi-token word

# avg_rank, avg_prob, ranks, probs = get_token_sequence_rank(model, messages, target_text)

# print(f"Average Rank: {avg_rank}")
# print(f"Geometric Mean Probability: {avg_prob:.4f}")
# print(f"Individual Token Ranks: {ranks}")
# print(f"Individual Token Probabilities: {probs}")


In [ ]:
from rich import print as rprint
from typing import Any, Callable, Dict, List, Optional, Tuple, Union
def test_prompt(
    prompt_tokens: torch.tensor , # (tokenized prompts)
    answer: Union[str, list[str]],
    model,  # Can't give type hint due to circular imports
    prepend_space_to_answer: bool = True,
    print_details: bool = True,
    prepend_bos: Optional[bool] = True,
    top_k: int = 10,
) -> None:
    """Test if the Model Can Give the Correct Answer to a Prompt.

    Intended for exploratory analysis. Prints out the performance on the answer (rank, logit, prob),
    as well as the top k tokens. Works for multi-token prompts and multi-token answers.

    Warning:

    This will print the results (it does not return them).

    Examples:

    >>> from transformer_lens import HookedTransformer, utils
    >>> model = HookedTransformer.from_pretrained("tiny-stories-1M")
    Loaded pretrained model tiny-stories-1M into HookedTransformer

    >>> prompt = "Why did the elephant cross the"
    >>> answer = "road"
    >>> utils.test_prompt(prompt, answer, model)
    Tokenized prompt: ['<|endoftext|>', 'Why', ' did', ' the', ' elephant', ' cross', ' the']
    Tokenized answer: [' road']
    Performance on answer token:
    Rank: 2        Logit: 14.24 Prob:  3.51% Token: | road|
    Top 0th token. Logit: 14.51 Prob:  4.59% Token: | ground|
    Top 1th token. Logit: 14.41 Prob:  4.18% Token: | tree|
    Top 2th token. Logit: 14.24 Prob:  3.51% Token: | road|
    Top 3th token. Logit: 14.22 Prob:  3.45% Token: | car|
    Top 4th token. Logit: 13.92 Prob:  2.55% Token: | river|
    Top 5th token. Logit: 13.79 Prob:  2.25% Token: | street|
    Top 6th token. Logit: 13.77 Prob:  2.21% Token: | k|
    Top 7th token. Logit: 13.75 Prob:  2.16% Token: | hill|
    Top 8th token. Logit: 13.64 Prob:  1.92% Token: | swing|
    Top 9th token. Logit: 13.46 Prob:  1.61% Token: | park|
    Ranks of the answer tokens: [(' road', 2)]

    Args:
        prompt:
            The prompt string, e.g. "Why did the elephant cross the".
        answer:
            The answer, e.g. "road". Note that if you set prepend_space_to_answer to False, you need
            to think about if you have a space before the answer here (as e.g. in this example the
            answer may really be " road" if the prompt ends without a trailing space). If this is a
            list of strings, then we only look at the next-token completion, and we compare them all
            as possible model answers.
        model:
            The model.
        prepend_space_to_answer:
            Whether or not to prepend a space to the answer. Note this will only ever prepend a
            space if the answer doesn't already start with one.
        print_details:
            Print the prompt (as a string but broken up by token), answer and top k tokens (all
            with logit, rank and probability).
        prepend_bos:
            Overrides self.cfg.default_prepend_bos if set. Whether to prepend
            the BOS token to the input (applicable when input is a string). Models generally learn
            to use the BOS token as a resting place for attention heads (i.e. a way for them to be
            "turned off"). This therefore often improves performance slightly.
        top_k:
            Top k tokens to print details of (when print_details is set to True).

    Returns:
        None (just prints the results directly).
    """
    tokenizer = model.tokenizer
    model = model.model
    answers = [answer] if isinstance(answer, str) else answer
    n_answers = len(answers)
    using_multiple_answers = n_answers > 1

    if prepend_space_to_answer:
        answers = [answer if answer.startswith(" ") else " " + answer for answer in answers]

    # GPT-2 often treats the first token weirdly, so lets give it a resting position
    # prompt_tokens = model.to_tokens(prompt, prepend_bos=prepend_bos)

    # prompt_tokens = tokenizer(prompt, add_special_tokens=False)["input_ids"]
    # Prepend BOS token if available
    if prepend_bos and tokenizer.bos_token_id is not None:
        prompt_tokens = [tokenizer.bos_token_id] + prompt_tokens

    
    # answer_tokens = model.to_tokens(answers, prepend_bos=False)


    answer_tokens = tokenizer(answers, add_special_tokens=False)["input_ids"]
    # Prepend BOS token if available
    if prepend_bos and tokenizer.bos_token_id is not None:
        answer_tokens = [tokenizer.bos_token_id] + answer_tokens
    answer_tokens = torch.tensor(answer_tokens)
    prompt_tokens = torch.tensor(prompt_tokens)
    # If we have multiple answers, we're only allowed a single token generation
    if using_multiple_answers:
        answer_tokens = answer_tokens[:, :1]

    # Deal with case where answers is a list of strings
    prompt_tokens = prompt_tokens.repeat(answer_tokens.shape[0], 1)
    tokens = torch.cat((prompt_tokens, answer_tokens), dim=1)
    # temp_tokens = tokenizer(prompt_tokens, add_special_tokens=False)["input_ids"]
    # Prepend BOS token if available
    if prepend_bos and tokenizer.bos_token_id is not None:
        prompt_tokens = [tokenizer.bos_token_id] + prompt_tokens
    
    # Convert token IDs to a list of string tokens
    prompt_str_tokens = tokenizer.convert_ids_to_tokens(prompt_tokens[0])
    # prompt_str_tokens = model.to_str_tokens(prompt, prepend_bos=prepend_bos)


    temp_tokens = [tokenizer(answer, add_special_tokens=False)["input_ids"] for answer in answers]
    answer_str_tokens_list = [tokenizer.convert_ids_to_tokens(temp) for temp in temp_tokens]
    prompt_length = len(prompt_str_tokens)
    answer_length = 1 if using_multiple_answers else len(answer_str_tokens_list[0])
    if print_details:
        print("Tokenized prompt:", prompt_str_tokens)
        if using_multiple_answers:
            print("Tokenized answers:", answer_str_tokens_list)
        else:
            print("Tokenized answer:", answer_str_tokens_list[0])
    tokens = tokens.to(device)
    logits = model(tokens).logits
    probs = logits.softmax(dim=-1)
    answer_ranks = []

    for index in range(prompt_length, prompt_length + answer_length):
        # Get answer tokens for this sequence position
        answer_tokens = tokens[:, index]
        answer_str_tokens = [a[index - prompt_length] for a in answer_str_tokens_list]
        # Offset by 1 because models predict the NEXT token
        token_probs = probs[:, index - 1]
        sorted_token_probs, sorted_token_positions = token_probs.sort(descending=True)
        answer_token_ranks = sorted_token_positions.argsort(-1)[
            range(n_answers), answer_tokens.cpu()
        ].tolist()
        answer_ranks.append(
            [
                (answer_str_token, answer_token_rank)
                for answer_str_token, answer_token_rank in zip(
                    answer_str_tokens, answer_token_ranks
                )
            ]
        )
        if print_details:
            # String formatting syntax - the first number gives the number of characters to pad to, the second number gives the number of decimal places.
            # rprint gives rich text printing
            rprint(
                f"Performance on answer token{'s' if n_answers > 1 else ''}:\n"
                + "\n".join(
                    [
                        f"[b]Rank: {answer_token_ranks[i]: <8} Logit: {logits[i, index-1, answer_tokens[i]].item():5.2f} Prob: {token_probs[i, answer_tokens[i]].item():6.2%} Token: |{answer_str_tokens[i]}|[/b]"
                        for i in range(n_answers)
                    ]
                )
            )
            for i in range(top_k):
                print(
                    f"Top {i}th token. Logit: {logits[0, index-1, sorted_token_positions[0, i]].item():5.2f} Prob: {sorted_token_probs[0, i].item():6.2%} Token: |{tokenizer.decode(sorted_token_positions[0, i])}|"
                )

    # If n_answers = 1 then unwrap answer ranks, so printed output matches original version of function
    if not using_multiple_answers:
        single_answer_ranks = [r[0] for r in answer_ranks]
        rprint(f"[b]Ranks of the answer tokens:[/b] {single_answer_ranks}")
        return single_answer_ranks
    else:
        rprint(f"[b]Ranks of the answer tokens:[/b] {answer_ranks}")
        return answer_ranks

In [ ]:
# prompts = ["What is the capital city of England?"]

# messages = [
#             {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
#             {"role": "user", "content": prompts[0]}
#         ]

# # model.tokenizer.decode(sorted_token_positions[0, i])
# text = model.tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True
# )
# prompt = model.tokenizer(text, return_tensors="pt")["input_ids"]
# test_prompt(prompt, " The capital city of England is London", model)

In [ ]:
# ranks = test_prompt(prompt, " The capital city of England is London", model)

In [ ]:
# x = ranks.copy()
# x

# Dataset

TODO BRO

In [ ]:
# from datasets import load_dataset

# ds = load_dataset("coastalcph/mpararel")


In [ ]:
# data = (ds['train'])

In [ ]:
# languages = {'en', 'it', 'fr', 'sp', 'zh'}  # Set of desired languages

# ds_filtered30 = ds['train'].filter(lambda example: example['language'] in languages and 
#                                                   example['relation'] == 'P30' and 
#                                                   example['template_id'] == 0)


In [ ]:
# for i in ds_filtered30.select(range(30)):
#     print(i)

In [ ]:
# for i in range (3):
#     print(ds[i])

# Evaluate

In [ ]:
model.model.config.num_hidden_layers

In [ ]:
import torch.nn.functional as F
from functools import reduce



def calc_logprob_answer(text, answer, model):
    answer = f" {answer}" if not answer.startswith(" ") else answer
    filled_sentence = text + answer
    filled_input_ids = model.tokenizer(filled_sentence, return_tensors="pt")["input_ids"].to(device)
    str_tokens = model.tokenizer.convert_ids_to_tokens(filled_input_ids[0])
    # print(f"str_tokens text: {str_tokens}")
    with torch.no_grad():
        outputs = model.model(input_ids=filled_input_ids)
        logits = outputs.logits  # Shape: (batch_size, seq_len, vocab_size)
    log_probs = F.log_softmax(logits, dim=-1)
    seq_log_probs = 0
    for i in range(1, filled_input_ids.size(1)):  
        token_id = filled_input_ids[0, i]
        seq_log_probs += log_probs[0, i-1, token_id].item()
    # print(f"seq_log_probs: {seq_log_probs}")
    return seq_log_probs

def calculate_logprob_diff(model, is_intervening, intervention_data, lang_target, lang_baseline, lape_langs, operation_target, range_layers, operation_non_target="*1"):
    """
    calculate the log prob diff of a language intervention scenario.
    For example: for a baseline en and target it, calculate the logprob(answer_it) - logprob(answer_en)
    for each prompt

    Output: 
    [(question_id, logprob_diff_answer_target_baseline)]
    e.g.
    (question_id, logprob(" Londra") - logprob(" London"))
    """
    
    logprob_diff_lang = []
    tokenizer = model.tokenizer

    #TODO REGISTER HOOK ON LANG TARGET ONLY IF LANG TARGET != LANG BASELINE
    num_layers = model.model.config.num_hidden_layers
    
    if model.model_name == 'Qwen/Qwen2.5-0.5B-Instruct' or model.model_name == 'Qwen/Qwen2.5-3B-Instruct':
        for data in intervention_data:
            prompt = data['query'][lang_baseline]
            
            messages = [
                {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
            text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            answer_target = data['answers'][lang_target]
            answer_baseline = data['answers'][lang_baseline]


            logprob_answer_target = 0
            logprob_answer_baseline = 0
            if is_intervening:
                clean_hooks(model)
                handlers = []
                assert range_layers[-1] <= num_layers
                for i in (range_layers):
                    mlp = model.model.model.layers[i].mlp
                    handlers.append(mlp.act_fn.register_forward_hook(
                        set_activation_mlp_v2(replace_method="lape", model_name=model.model_name, name=f"{i}",
                          lape_langs=lape_langs, target_lang=lang_target, operation_non_target=operation_non_target, 
                          operation_target=operation_target)))
                logprob_answer_target = calc_logprob_answer(text, answer_target, model)
                logprob_answer_baseline= calc_logprob_answer(text, answer_baseline, model)
                for i in handlers:
                    i.remove()
                clean_hooks(model)
            else: 
                logprob_answer_target = calc_logprob_answer(text, answer_target, model)
                logprob_answer_baseline= calc_logprob_answer(text, answer_baseline, model)
            log_prob_diff = logprob_answer_target-logprob_answer_baseline
            question_id = data['template_id']
            logprob_diff_lang.append((question_id, log_prob_diff))
    else:
        print("MODEL NOT AVAILABLE!")
    return logprob_diff_lang

def calculate_diff_and_ratio(logprob_diff_baseline, logprob_diff_target):
    baseline_dict = dict(logprob_diff_baseline)
    target_dict = dict(logprob_diff_target)
    diff_of_diff = [
        (qid, target_dict[qid] - baseline_dict[qid]) 
        for qid in baseline_dict.keys() & target_dict.keys()
    ]
    ratio_of_diff = [
        (qid, abs(target_dict[qid] / baseline_dict[qid]) if baseline_dict[qid] != 0 else float('inf')) 
        for qid in baseline_dict.keys() & target_dict.keys()
    ]
    return diff_of_diff, ratio_of_diff
def eval_log_probs(model, intervention_data, lang_baseline, lang_target, lape_langs, operation_target, range_layers, return_df=False):
    """
    INPUT:
        model: HookTransformer model
        intervention_data : list of dictionary, e.g.
            [{
            'template_id' : 'question_id',
            'query': 'question', 
            'answers' : 
                {'en' : 'answer_en', 
                'id' : 'answer_id'
                
                }}]
        lang_baseline: str lang baseline e.g. 'en'
        lang_target: str lang target e.g. 'it'
        
    OUTPUT: beururtan sama template_id
        log_prob_diff = 
        {
            'lang_baseline': [(question_id, log_prob_diff_answer_target_baseline_q1), (question_id, log_prob_diff_answer_target_baseline_q2),... ],
            'lang_target': [(question_id, log_prob_diff_answer_target_baseline_q1), (question_id, log_prob_diff_answer_target_baseline_q2),... ],
        },
        df (Optional)
    """
    logprob_diff_baseline = calculate_logprob_diff(model, False, intervention_data, lang_target, lang_baseline, lape_langs, operation_target, range_layers)
    logprob_diff_target = calculate_logprob_diff(model, True, intervention_data, lang_target, lang_baseline, lape_langs, operation_target, range_layers)
    if return_df:
        df_logprob_diff_baseline = pd.DataFrame(logprob_diff_baseline, columns=["qid", "logprob_baseline_diff"])
        df_logprob_diff_target = pd.DataFrame(logprob_diff_target, columns=["qid", "logprob_diff_target"])
        data = []
        for item in intervention_data:
            qid = item["template_id"]
            question = item['query'].get(lang_baseline, None)
            answer_baseline = item["answers"].get(lang_baseline, None)  # English (en-US)
            answer_target = item["answers"].get(lang_target, None)   # French (fr-FR)
            data.append((qid, question, answer_baseline, answer_target))
        df_answers = pd.DataFrame(data, columns=["qid", f"question {languages[lang_baseline]}", f"answer_baseline {languages[lang_baseline]}", f"answer_target {languages[lang_target]}"])
        diff_of_diff, ratio_of_diff = calculate_diff_and_ratio(logprob_diff_baseline, logprob_diff_target)
        # print(diff_of_diff)
        # print(f"situ: {logprob_diff_baseline}")
        # print(f"situ: {diff_of_diff}")

        df_diff_of_diff = pd.DataFrame(diff_of_diff, columns=["qid", "diff_of_diff"])
        df_ratio_of_diff = pd.DataFrame(ratio_of_diff, columns=["qid", "ratio_of_diff"])
        dfs = [df_answers, df_logprob_diff_baseline, df_logprob_diff_target, df_diff_of_diff, df_ratio_of_diff]
        
        df_final = reduce(lambda left, right: pd.merge(left, right, on="qid", how="outer"), dfs)
        # print(df_final)
        return logprob_diff_target, logprob_diff_baseline, df_final
        
    return logprob_diff_target, logprob_diff_baseline

def beautify_df(df, lang_target, lang_baseline):
    df_display = df.rename(columns={
    "qid": f"qid in {lang_baseline}",
    "logprob_baseline_diff": f"logprob {lang_target} answer - {lang_baseline} answer in baseline",
    "logprob_diff_target": f"logprob {lang_target} answer - {lang_baseline} answer in {lang_target} intervention",
    "diff_of_diff": f"Difference of Differences {lang_target} - {lang_baseline}",
    "ratio_of_diff": f"Ratio of Differences {lang_target} - {lang_baseline}",
    "answer_baseline": f"Baseline Answer in {lang_baseline}",
    "answer_target": f"Target Answer in {lang_target}"
})
    return df_display
    

In [ ]:
# from collections import defaultdict
# import random

# # Define the languages you want (assuming these are the ones in the dataset)
# languages = {'en', 'it', 'fr', 'es', 'zh'}  # Using 'es' for Spanish instead of 'sp'

# # Filter dataset to include only relations P30 and P36
# ds_filtered = ds['train'].filter(lambda example: example['language'] in languages and 
#                                                  example['relation'] in {'P30', 'P36'} and 
#                                                  example['template_id'] == 0)

# # Group by template_id
# grouped_by_template = defaultdict(list)
# for example in ds_filtered:
#     grouped_by_template[example['template_id']].append(example)

# # Ensure unique template IDs and 5 languages per question
# intervention_data = []
# used_qids = set()

# # Shuffle the template IDs for randomness
# template_ids = list(grouped_by_template.keys())
# random.shuffle(template_ids)

# for template_id in template_ids:
#     examples = grouped_by_template[template_id]
    
#     # Ensure we have enough languages (at least 5)
#     language_map = {ex['language']: ex for ex in examples if ex['language'] in languages}
    
#     if len(language_map) == 5:
#         qid = examples[0]['id']  # Get a unique QID from one of the examples
#         if qid not in used_qids:
#             used_qids.add(qid)
            
#             # Construct query-answer pairs
#             query = {lang_code: ex['question'] for lang_code, ex in language_map.items()}
#             answer = {lang_code: ex['answer'] for lang_code, ex in language_map.items()}

#             intervention_data.append({
#                 "template_id": template_id,
#                 "query": query,
#                 "answers": answer
#             })
    
#     if len(intervention_data) >= 50:
#         break  # Stop once we have 50 unique questions

# # Print a sample to verify
# print(intervention_data[:3])  # Show first 3 samples for validation


In [ ]:
intervention_data = [
    {
        "template_id": 1,
        "query": {
            10: "What is the hardest natural substance on Earth?",
            21: "Qual è la sostanza naturale più dura sulla Terra?",
            19: "Apa zat alami terkeras di Bumi?",
            14: "Quelle est la substance naturelle la plus dure sur Terre?",
            11: "¿Cuál es la sustancia natural más dura de la Tierra?"
        },
        "answers": {
            10: "Diamond",
            21: "Diamante",
            19: "Berlian",
            14: "Diamant",
            11: "Diamante"
        }
    },
    {
        "template_id": 2,
        "query": {
            10: "What is the fastest land animal?",
            21: "Qual è l'animale terrestre più veloce?",
            19: "Apa hewan darat tercepat?",
            14: "Quel est l'animal terrestre le plus rapide?",
            11: "¿Cuál es el animal terrestre más rápido?"
        },
        "answers": {
            10: "Cheetah",
            21: "Ghepardo",
            19: "Cheetah",
            14: "Guépard",
            11: "Guepardo"
        }
    },
    {
        "template_id": 3,
        "query": {
            10: "What is the first element on the periodic table?",
            21: "Qual è il primo elemento della tavola periodica?",
            19: "Apa unsur pertama dalam tabel periodik?",
            14: "Quel est le premier élément du tableau périodique?",
            11: "¿Cuál es el primer elemento de la tabla periódica?"
        },
        "answers": {
            10: "Hydrogen",
            21: "Idrogeno",
            19: "Hidrogen",
            14: "Hydrogène",
            11: "Hidrógeno"
        }
    },
    {
        "template_id": 4,
        "query": {
            10: "How many continents are there on Earth?",
            21: "Quanti continenti ci sono sulla Terra?",
            19: "Ada berapa benua di Bumi?",
            14: "Combien y a-t-il de continents sur Terre?",
            11: "¿Cuántos continentes hay en la Tierra?"
        },
        "answers": {
            10: "Seven",
            21: "Sette",
            19: "Tujuh",
            14: "Sept",
            11: "Siete"
        }
    },
    {
        "template_id": 5,
        "query": {
            10: "What is the powerhouse of the cell?",
            21: "Qual è la centrale energetica della cellula?",
            19: "Apa pusat tenaga dalam sel?",
            14: "Quelle est la centrale énergétique de la cellule?",
            11: "¿Cuál es la central energética de la célula?"
        },
        "answers": {
            10: "Mitochondria",
            21: "Mitocondrio",
            19: "Mitokondria",
            14: "Mitochondrie",
            11: "Mitocondria"
        }
    },
    {
        "template_id": 6,
        "query": {
            10: "Which organelle is responsible for photosynthesis?",
            21: "Quale organello è responsabile della fotosintesi?",
            19: "Organel mana yang bertanggung jawab atas fotosintesis?",
            14: "Quel organite est responsable de la photosynthèse?",
            11: "¿Qué orgánulo es responsable de la fotosíntesis?"
        },
        "answers": {
            10: "Chloroplast",
            21: "Cloroplasto",
            19: "Kloroplas",
            14: "Chloroplaste",
            11: "Cloroplasto"
        }
    },
    {
        "template_id": 7,
        "query": {
            10: "What is the basic unit of life?",
            21: "Qual è l'unità di base della vita?",
            19: "Apa unit dasar kehidupan?",
            14: "Quelle est l'unité de base de la vie?",
            11: "¿Cuál es la unidad básica de la vida?"
        },
        "answers": {
            10: "Cell",
            21: "Cellula",
            19: "Sel",
            14: "Cellule",
            11: "Célula"
        }
    },
    {
        "template_id": 8,
        "query": {
            10: "What is the biggest animal on land?",
            21: "Qual è il più grande animale terrestre?",
            19: "Apa hewan terbesar di darat?",
            14: "Quel est le plus grand animal terrestre?",
            11: "¿Cuál es el animal más grande en tierra?"
        },
        "answers": {
            10: "African elephant",
            21: "Elefante africano",
            19: "Gajah Afrika",
            14: "Éléphant d'Afrique",
            11: "Elefante africano"
        }
    },
    {
        "template_id": 9,
        "query": {
            10: "What is the coldest continent in the world?",
            21: "Qual è il continente più freddo del mondo?",
            19: "Apa benua terdingin di dunia?",
            14: "Quel est le continent le plus froid du monde?",
            11: "¿Cuál es el continente más frío del mundo?"
        },
        "answers": {
            10: "Antarctica",
            21: "Antartide",
            19: "Antartika",
            14: "Antarctique",
            11: "Antártida"
        }
    },
    {
        "template_id": 10,
        "query": {
            10: "What season does Australia experience in December?",
            21: "Quale stagione vive l'Australia a dicembre?",
            19: "Musim apa yang dialami Australia pada bulan Desember?",
            14: "Quelle saison l'Australie vit-elle en décembre?",
            11: "¿Qué estación experimenta Australia en diciembre?"
        },
        "answers": {
            10: "Summer",
            21: "Estate",
            19: "Musim panas",
            14: "Été",
            11: "Verano"
        }
    },
    {
        "template_id": 11,
        "query": {
            10: "What is the proper geographical term for a cluster or chain of islands?",
            21: "Qual è il termine geografico corretto per un gruppo o una catena di isole?",
            19: "Apa istilah geografis yang benar untuk sekelompok atau rantai pulau?",
            14: "Quel est le terme géographique approprié pour un groupe ou une chaîne d'îles?",
            11: "¿Cuál es el término geográfico adecuado para un grupo o cadena de islas?"
        },
        "answers": {
            10: "Archipelago",
            21: "Arcipelago",
            19: "Kepulauan",
            14: "Archipel",
            11: "Archipiélago"
        }
    }
    
]


In [ ]:
languages_reversed = {v: k for k, v in languages.items()}


## LAPE 2%

In [ ]:
lape_massive_qwen_2 = visualize_lape(full_languages_raw_values, 24, top_rate = 0.02,filter_rate=0.80,activation_bar_ratio=0.80)
lape_langs = {key:lape_massive_qwen_2[key] for key, value in enumerate(lape_massive_qwen_2)}


## Difference of Difference (DoD) > 0 → The first difference is larger than the second.
## |Ratio of Difference (RoD)| > 1 → The first difference is proportionally larger than the second.

In [ ]:
from IPython.display import display


def eval_intervention_a_baselang(intervention_data, lang_baseline, lang_targets, operation_target, range_layers, lape_langs, return_df=True, save_each_df = False):
    diff_dict = {key:None for key in lang_targets}
    for lang_target in lang_targets:
        id_lang_baseline = languages_reversed[lang_baseline]
        id_lang_target = languages_reversed[lang_target]
        diff_target, diff_baseline, df = eval_log_probs(
            model, intervention_data, 
            lang_baseline=id_lang_baseline, 
            lang_target=id_lang_target, lape_langs=lape_langs, 
            operation_target=operation_target, range_layers=range_layers, 
            return_df=return_df)
        # count average diff of diff and ratio of diff
        # print(f"sini: {diff_baseline}")
        diff_of_diff, ratio_of_diff = calculate_diff_and_ratio(diff_baseline, diff_target)
        # print(f"sini: {diff_of_diff}")
        positive_count_dod = sum(1 for _, diff in diff_of_diff if diff > 0)
        # print(positive_count_dod)
        average_dod = positive_count_dod / len(diff_of_diff) if len(diff_of_diff) > 0 else 0
        # print(average_dod)
        positive_count_rod = sum(1 for _, diff in ratio_of_diff if diff != float('inf') and diff > 1)
        average_rod = positive_count_rod / len(ratio_of_diff) if len(ratio_of_diff) > 0 else 0
        # print(positive_count_rod)
        # print(average_rod)
        
        diff_dict[lang_target] = (average_dod, average_rod)
        df_display =beautify_df(df, lang_target, lang_baseline)
        # df_display.head(20)
        # print(df_display)
        display(df_display)
        if save_each_df:
            df_display.to_csv(f"baseline_{lang_baseline}/{lang_target}.csv")
    # print(diff_dict)
    df_result = pd.DataFrame.from_dict(diff_dict, orient='index', columns=['Diff of diff percentage', 'Ratio of diff percentage']).reset_index()
    # Rename the columns properly
    df_result.rename(columns={'index': f'neuron intervention (baseline {lang_baseline})'}, inplace=True)
    display(df_result)
    return df_result

In [ ]:
#MODIFY  THIS ONLY 
lang_baseline = "Italian (it-IT)"
lang_targets = ['French (fr-FR)', "English (en-US)", "Indonesian (id-ID)", "Spanish (es-ES)"]
operation_target = "+2"
range_layers=[5,17,18,19,20,21,22,23]
lape_langs = lape_langs
#############
df_italy = eval_intervention_a_baselang(intervention_data, lang_baseline, lang_targets, operation_target, range_layers, lape_langs, return_df=True, save_each_df = False)
df_italy.to_csv("df_italy.csv")

In [ ]:
#MODIFY  THIS ONLY 
lang_baseline = "English (en-US)"
lang_targets = ['French (fr-FR)', "Indonesian (id-ID)", "Spanish (es-ES)", "Italian (it-IT)"]
operation_target = "+2"
range_layers=[5,17,18,19,20,21,22,23]
lape_langs = lape_langs
#############
df_english = eval_intervention_a_baselang(intervention_data, lang_baseline, lang_targets, operation_target, range_layers, lape_langs, return_df=True, save_each_df = False)

df_english.to_csv("df_english.csv")

In [ ]:
#MODIFY  THIS ONLY 
lang_baseline = "Indonesian (id-ID)" 
lang_targets = ['French (fr-FR)', "English (en-US)", "Spanish (es-ES)", "Italian (it-IT)"]
operation_target = "+2"
range_layers=[5,17,18,19,20,21,22,23]
lape_langs = lape_langs
#############
df_indo = eval_intervention_a_baselang(intervention_data, lang_baseline, lang_targets, operation_target, range_layers, lape_langs, return_df=True, save_each_df = False)
df_indo.to_csv("df_indo.csv")

In [ ]:
#MODIFY  THIS ONLY 
lang_baseline =  "Spanish (es-ES)"
lang_targets = ['French (fr-FR)', "English (en-US)", "Indonesian (id-ID)", "Italian (it-IT)"]
operation_target = "+2"
range_layers=[5,17,18,19,20,21,22,23]
lape_langs = lape_langs
#############
df_spanish = eval_intervention_a_baselang(intervention_data, lang_baseline, lang_targets, operation_target, range_layers, lape_langs, return_df=True, save_each_df = False)
df_spanish.to_csv("df_spanish.csv")

In [ ]:
#MODIFY  THIS ONLY 
lang_baseline =  'French (fr-FR)'
lang_targets = ["Spanish (es-ES)", "English (en-US)", "Indonesian (id-ID)", "Italian (it-IT)"]
operation_target = "+2"
range_layers=[5,17,18,19,20,21,22,23]
lape_langs = lape_langs
#############
df_french = eval_intervention_a_baselang(intervention_data, lang_baseline, lang_targets, operation_target, range_layers, lape_langs, return_df=True, save_each_df = False)
df_french.to_csv("df_french.csv")

# Free text questions Eval

### prompt hanya bahasa inggris

In [ ]:
languages_reversed

In [ ]:
questions_answers = [
    {
    "q":"My favourite food is", "answer_baseline" : "sandwich", 
        "answers_target" : {
            languages_reversed["French (fr-FR)"] : "croissant", 
            languages_reversed["Italian (it-IT)"]: "pizza"}
    },
    {
    "q":"My favourite football player is", "answer_baseline" : "David Beckham", 
        "answers_target" : {
            languages_reversed["French (fr-FR)"] : "Kylian Mbappe", 
            languages_reversed["Portuguese (pt-PT)"]: "Cristiano Ronaldo",
            languages_reversed["Dutch (nl-NL)"]: "Virgil Van Dijk"},
    },
    {
    "q":"My favourite city is", "answer_baseline" : "London", 
        "answers_target" : {
            languages_reversed["French (fr-FR)"] : "Paris", 
            languages_reversed["Indonesian (id-ID)"]: "Jakarta",
            languages_reversed["Chinese (Simplified) (zh-CN)"]: "北京"},
    },
    
    
                    
]

In [ ]:
def free_text_eval(model, questions_answers, lape_langs, operation_target, range_layers, operation_non_target="*1"):
    rank_list = []
    
    num_layers = model.model.config.num_hidden_layers
    for qna in questions_answers:
        q = qna['q']
        rank_langs = dict()
        # messages = [
        #         {"role": "system", "content": "Continue the sentence in a natural way."},
        #         {"role": "user", "content": q}
        #     ]
        text = q
        # model.tokenizer.decode(sorted_token_positions[0, i])
        # text = model.tokenizer.apply_chat_template(
        #     messages,
        #     tokenize=False,
        #     add_generation_prompt=True
        # )
        
        prompt = model.tokenizer(text, return_tensors="pt")["input_ids"]
        print(model.tokenizer.convert_ids_to_tokens(prompt[0]))
        print("\n🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸 BASELINE 🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸\n")
        
        rank_baseline = test_prompt(prompt, qna["answer_baseline"], model=model, top_k=20)
        rank_langs["baseline"] = rank_baseline
        # print(rank_langs)
        for lang_target,answer_target in qna['answers_target'].items():
            
            clean_hooks(model)
            handlers = []
            assert range_layers[-1] <= num_layers
            for i in (range_layers):
                mlp = model.model.model.layers[i].mlp
                handlers.append(mlp.act_fn.register_forward_hook(
                    set_activation_mlp_v2(replace_method="lape", model_name=model.model_name, name=f"{i}",
                      lape_langs=lape_langs, target_lang=lang_target, operation_non_target=operation_non_target, 
                      operation_target=operation_target)))
            print(f"\n🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸 INTVD ON {languages[lang_target]} 🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸🐸\n")
            
            rank_target = test_prompt(prompt, answer_target,  model=model, top_k=20)
            rank_langs[languages[lang_target]] = rank_target
            for i in handlers:
                i.remove()
            clean_hooks(model)
        # print(rank_langs)
        rank_list.append(rank_langs)
    return rank_list

In [ ]:
#MODIFY  THIS ONLY 
operation_target = "+2"
range_layers=[5,17,18,19,20,21,22,23]
#############
q0 = free_text_eval(model, questions_answers[:1], lape_langs, operation_target, range_layers, operation_non_target="*1")


In [ ]:
#MODIFY  THIS ONLY 
operation_target = "+5"
range_layers=range(24)
#############
q1 = free_text_eval(model, questions_answers[1:2], lape_langs, operation_target, range_layers, operation_non_target="*1")


In [ ]:
#MODIFY  THIS ONLY 
operation_target = "*5"
range_layers=range(24)
#############
q1 = free_text_eval(model, questions_answers[2:4], lape_langs, operation_target, range_layers, operation_non_target="*1")


# LAPE 5%

In [ ]:
lape_massive_qwen_5 = visualize_lape(full_languages_raw_values, 24, top_rate = 0.05,filter_rate=0.80,activation_bar_ratio=0.80)
lape_langs_5 = {key:lape_massive_qwen_2[key] for key, value in enumerate(lape_massive_qwen_5)}


In [ ]:
#MODIFY  THIS ONLY 
import os
folder_name = "/kaggle/working/lape5"

if not os.path.exists(folder_name):
    os.mkdir(folder_name)
lang_baseline = "Italian (it-IT)"
lang_targets = ['French (fr-FR)', "English (en-US)", "Indonesian (id-ID)", "Spanish (es-ES)"]
operation_target = "+2"
range_layers=[5,17,18,19,20,21,22,23]
lape_langs = lape_langs_5
#############
df_italy = eval_intervention_a_baselang(intervention_data, lang_baseline, lang_targets, operation_target, range_layers, lape_langs, return_df=True, save_each_df = False)
df_italy.to_csv("lape5/5_df_italy.csv")

In [ ]:
#MODIFY  THIS ONLY 
lang_baseline = "English (en-US)"
lang_targets = ['French (fr-FR)', "Indonesian (id-ID)", "Spanish (es-ES)", "Italian (it-IT)"]
operation_target = "+2"
range_layers=[5,17,18,19,20,21,22,23]
lape_langs = lape_langs_5
#############
df_english = eval_intervention_a_baselang(intervention_data, lang_baseline, lang_targets, operation_target, range_layers, lape_langs, return_df=True, save_each_df = False)

df_english.to_csv("lape5/5_df_english.csv")

In [ ]:
#MODIFY  THIS ONLY 
lang_baseline = "Indonesian (id-ID)" 
lang_targets = ['French (fr-FR)', "English (en-US)", "Spanish (es-ES)", "Italian (it-IT)"]
operation_target = "+2"
range_layers=[5,17,18,19,20,21,22,23]
lape_langs = lape_langs_5
#############
df_indo = eval_intervention_a_baselang(intervention_data, lang_baseline, lang_targets, operation_target, range_layers, lape_langs, return_df=True, save_each_df = False)
df_indo.to_csv("lape5/5_df_indo.csv")

In [ ]:

#MODIFY  THIS ONLY 
lang_baseline =  "Spanish (es-ES)"
lang_targets = ['French (fr-FR)', "English (en-US)", "Indonesian (id-ID)", "Italian (it-IT)"]
operation_target = "+2"
range_layers=[5,17,18,19,20,21,22,23]
lape_langs = lape_langs_5
#############
df_spanish = eval_intervention_a_baselang(intervention_data, lang_baseline, lang_targets, operation_target, range_layers, lape_langs, return_df=True, save_each_df = False)
df_spanish.to_csv("lape5/5_df_spanish.csv")

In [ ]:
#MODIFY  THIS ONLY 
lang_baseline =  'French (fr-FR)'
lang_targets = ["Spanish (es-ES)", "English (en-US)", "Indonesian (id-ID)", "Italian (it-IT)"]
operation_target = "+2"
range_layers=[5,17,18,19,20,21,22,23]
lape_langs = lape_langs_5
#############
df_french = eval_intervention_a_baselang(intervention_data, lang_baseline, lang_targets, operation_target, range_layers, lape_langs, return_df=True, save_each_df = False)
df_french.to_csv("lape5/5_df_french.csv")


free text

In [ ]:
lape_langs = lape_langs_5

#MODIFY  THIS ONLY 
operation_target = "+2"
range_layers=[5,17,18,19,20,21,22,23]
#############
q0 = free_text_eval(model, questions_answers[:1], lape_langs, operation_target, range_layers, operation_non_target="*1")




In [ ]:
lape_langs = lape_langs_5


#MODIFY  THIS ONLY 
operation_target = "+10"
range_layers=[5,17,18,19,20,21,22,23]
#############
q0 = free_text_eval(model, questions_answers[1:2], lape_langs, operation_target, range_layers, operation_non_target="*1")




In [ ]:
lape_langs = lape_langs_5

#MODIFY  THIS ONLY 
operation_target = "+10"
range_layers=[5,17,18,19,20,21,22,23]
#############
q0 = free_text_eval(model, questions_answers[2:3], lape_langs, operation_target, range_layers, operation_non_target="*1")
